# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,qaqortoq,60.7167,-46.0333,3.58,98,100,0.99,GL,2024-06-08
1,1,west island,-12.1568,96.8225,26.99,74,75,3.60,CC,2024-06-08
2,2,lompoc,34.6391,-120.4579,22.66,91,100,5.66,US,2024-06-08
3,3,chibougamau,49.9168,-74.3659,18.79,63,100,3.09,CA,2024-06-08
4,4,cockburn town,21.4612,-71.1419,28.03,83,100,6.33,TC,2024-06-08


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City", 
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
cities_narrow_lats_df = city_data_df[(city_data_df['Lat'] >= -19) & (city_data_df['Lat'] <= 40)]

# Drop any rows with null values
ideal_weather_city_df = cities_narrow_lats_df.dropna()

# Display sample data
ideal_weather_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,west island,-12.1568,96.8225,26.99,74,75,3.60,CC,2024-06-08
2,2,lompoc,34.6391,-120.4579,22.66,91,100,5.66,US,2024-06-08
4,4,cockburn town,21.4612,-71.1419,28.03,83,100,6.33,TC,2024-06-08
7,7,afaahiti,-17.7500,-149.2833,23.93,85,99,7.95,PF,2024-06-08
8,8,sanming,26.2486,117.6186,22.08,98,35,0.71,CN,2024-06-08
...,...,...,...,...,...,...,...,...,...,...
568,568,recani,31.2846,-4.2688,34.41,6,75,12.30,MA,2024-06-08
570,570,soio,-6.1349,12.3689,23.00,83,25,3.19,AO,2024-06-08
571,571,jiuquan,39.7432,98.5174,21.40,19,80,2.58,CN,2024-06-08
573,573,tartouss,34.8890,35.8866,25.71,77,0,1.97,SY,2024-06-08


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_city_df[['City', 'Country','Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,west island,CC,-12.1568,96.8225,74,
2,lompoc,US,34.6391,-120.4579,91,
4,cockburn town,TC,21.4612,-71.1419,83,
7,afaahiti,PF,-17.7500,-149.2833,85,
8,sanming,CN,26.2486,117.6186,98,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
limit = 20
radius = 10000
params = {
    "limit":limit,
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
        
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
west island - nearest hotel: Cocos Village Bungalows
lompoc - nearest hotel: Red Roof Inn Lompoc
cockburn town - nearest hotel: The Salt Raker Inn
afaahiti - nearest hotel: Omati Lodge
sanming - nearest hotel: 三明客家宾馆
freetown - nearest hotel: Formal Leona hotel
gao - nearest hotel: No hotel found
denia - nearest hotel: Hostal Comercio
anaco - nearest hotel: Hotel Muñíz
georgetown - nearest hotel: Page 63 hostel
la'ie - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
manaquiri - nearest hotel: No hotel found
sitangkai - nearest hotel: No hotel found
ryotsu-minato - nearest hotel: ゆたかや旅館
sinabang - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
shinas - nearest hotel: No hotel found
saipan - nearest hotel: Chalan Kanoa Beach Hotel
wenchang - nearest hotel: No hotel found
abepura - nearest hotel: Yoyo
laojunmiao - nearest hotel: No hotel found
aljezur - nearest hotel: Vicentina Hotel
at

,City,Country,Lat,Lng,Humidity,Hotel Name
1,west island,CC,-12.1568,96.8225,74,Cocos Village Bungalows
2,lompoc,US,34.6391,-120.4579,91,Red Roof Inn Lompoc
4,cockburn town,TC,21.4612,-71.1419,83,The Salt Raker Inn
7,afaahiti,PF,-17.7500,-149.2833,85,Omati Lodge
8,sanming,CN,26.2486,117.6186,98,三明客家宾馆
...,...,...,...,...,...,...
568,recani,MA,31.2846,-4.2688,6,Hotel Merzouga
570,soio,AO,-6.1349,12.3689,83,Hotel Maria Teres
571,jiuquan,CN,39.7432,98.5174,19,民族宾馆
573,tartouss,SY,34.8890,35.8866,77,فندق انترادوس


In [10]:
# Export the City_Data into a csv
hotel_df.to_csv("output_data/hotel.csv", index_label="City_ID")

In [11]:
# Read saved data
hotel_df = pd.read_csv("output_data/hotel.csv", index_col="City_ID")

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
1,west island,CC,-12.1568,96.8225,74,Cocos Village Bungalows
2,lompoc,US,34.6391,-120.4579,91,Red Roof Inn Lompoc
4,cockburn town,TC,21.4612,-71.1419,83,The Salt Raker Inn
7,afaahiti,PF,-17.7500,-149.2833,85,Omati Lodge
8,sanming,CN,26.2486,117.6186,98,三明客家宾馆


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Hotel Name",
    scale = 1,
    color = "City", 
)

# Display the map
hotel_map 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name)